# Golden Standard Parity Validation

## Purpose

This notebook validates that our Python analysis pipeline produces **identical results** to the legacy C++ analyzer (ffmm framework) when processing the same raw measurement data.

**Success criterion**: All harmonics must match to within numerical precision (ideally < 1e-9 relative error).

## Theory Background

### Rotating Coil Measurement Principle

A rotating coil measures magnetic field harmonics by rotating a coil inside a magnet aperture. The induced voltage is integrated to obtain flux, which is then Fourier-transformed to extract harmonics.

### Key Formulas (from ffmm C++ and Bottura PDF)

1. **Drift Correction** (legacy C++ style):
   ```
   flux = cumsum(df - mean(df)) - mean(cumsum(df))
   ```
   This removes DC offset from the incremental signal and centers the integrated flux.

2. **FFT Normalization**:
   ```
   f_n = 2 * FFT(flux)[n] / N_samples
   ```
   The factor of 2 accounts for the two-sided FFT spectrum.

3. **Kn Calibration** (coil sensitivity):
   ```
   C_n = f_n / conj(kn) * Rref^(n-1)
   ```
   Where `kn` are complex calibration coefficients from the measurement head geometry.

4. **Rotation** (phase alignment to main field):
   ```
   C_n_rotated = C_n * exp(-i * (n-m) * angle_m)
   ```
   Where `m` is the magnet order (2 for quadrupole) and `angle_m = arg(C_m)`.

### Harmonic Merge Strategy

- **Absolute channel (ABS)**: Direct measurement, used for main field (n ≤ m)
- **Compensated channel (CMP)**: Bucking coil cancels main field, used for errors (n > m)

For a quadrupole (m=2): Use ABS for n=1,2 and CMP for n=3,4,...,15

---
## 1. Configuration

Edit this cell to point to your golden standard dataset.

In [ ]:
from pathlib import Path

# =============================================================================
# DATASET CONFIGURATION - Edit these paths for your dataset
# =============================================================================

# Path to the measurement folder (contains flux files, current files, results)
DATASET_FOLDER = Path("../../golden_standards/golden_standard_01_LIU_BTP8/Central/20190718_141359_LIU")

# Path to the Kn calibration file (must match the compensation scheme used)
# BD_AE = B-D absolute, A-E compensated (for quadrupole with 4-coil bucking)
KN_FILE = Path("../../golden_standards/golden_standard_01_LIU_BTP8/COIL_PCB/PCB_DQ_5_18_7_250_47x50_Hall/Kn-Th/Kn_DQ_5_18_7_250_47x50_0001_BD_AE.txt")

# =============================================================================
# MAGNET PARAMETERS - From the Parameters.txt file
# =============================================================================

MAGNET_ORDER = 2      # 1=dipole, 2=quadrupole, 3=sextupole, etc.
R_REF_M = 1.0         # Reference radius in meters (affects multipole scaling)
L_COIL_M = 1.32209    # Coil length in meters
SHAFT_SPEED_RPM = 60  # Rotation speed (absolute value)

# =============================================================================
# CRITICAL: Samples per turn
# =============================================================================
# This value must match how the data was acquired!
# For BTP8 format: typically 512 samples per revolution
# Using the wrong value causes ~60x magnitude errors!

SAMPLES_PER_TURN = 512  # CORRECT for BTP8

# =============================================================================
# ANALYSIS OPTIONS - Must match the reference (from Options column)
# =============================================================================

OPTIONS = ("dri", "rot")  # dri=drift correction, rot=rotation to main field

# Output directory for comparison reports
OUTPUT_DIR = Path("../../outputs/golden_runs/LIU_BTP8_20190718_141359")

print("Configuration loaded successfully.")
print(f"  Magnet order: {MAGNET_ORDER} ({'dipole' if MAGNET_ORDER==1 else 'quadrupole' if MAGNET_ORDER==2 else 'higher'})")
print(f"  Samples/turn: {SAMPLES_PER_TURN}")
print(f"  Options: {OPTIONS}")

---
## 2. Setup and Imports

In [ ]:
import sys
import json
from datetime import datetime, timezone
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add repo to path
repo_root = Path("../..").resolve()
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

# Core analysis imports
from rotating_coil_analyzer.analysis.kn_pipeline import (
    load_segment_kn_txt,
    compute_legacy_kn_per_turn,
    merge_coefficients,
)

print("All imports successful.")

In [ ]:
# Resolve all paths
notebook_dir = Path(".").resolve()
dataset_folder = (notebook_dir / DATASET_FOLDER).resolve()
kn_file = (notebook_dir / KN_FILE).resolve()
output_dir = (notebook_dir / OUTPUT_DIR).resolve()
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Dataset folder: {dataset_folder}")
print(f"  Exists: {dataset_folder.exists()}")
print(f"Kn file: {kn_file}")
print(f"  Exists: {kn_file.exists()}")

if not dataset_folder.exists():
    raise FileNotFoundError(f"Dataset folder not found: {dataset_folder}")
if not kn_file.exists():
    raise FileNotFoundError(f"Kn file not found: {kn_file}")

---
## 3. Load Reference Results

The golden reference results were produced by the legacy C++ analyzer (ffmm framework).
Our goal is to reproduce these exact values.

In [ ]:
# Find the reference results file
ref_files = list(dataset_folder.glob("*results*.txt"))
ref_files = [f for f in ref_files if "Average" not in f.name and "Parameters" not in f.name]

if not ref_files:
    raise FileNotFoundError("No reference results file found in dataset folder")

ref_path = ref_files[0]
print(f"Loading reference: {ref_path.name}")

ref_df = pd.read_csv(ref_path, sep="\t")
print(f"\nReference data shape: {ref_df.shape}")
print(f"Number of turns: {len(ref_df)}")

# Show column names (these tell us what harmonics are available)
harmonic_cols = [c for c in ref_df.columns if c.startswith('B') or c.startswith('A')]
print(f"\nHarmonic columns: {harmonic_cols[:10]}...") 

In [ ]:
# Understand the reference data structure
print("Reference data summary:")
print(f"  Total measurement turns: {len(ref_df)}")
print(f"  Current range: [{ref_df['I FGC(A)'].min():.1f}, {ref_df['I FGC(A)'].max():.1f}] A")

# Main field range (B2 for quadrupole)
main_col = f"B{MAGNET_ORDER} (T)"
if main_col in ref_df.columns:
    print(f"  {main_col} range: [{ref_df[main_col].min():.6f}, {ref_df[main_col].max():.6f}] T")

# Show first few rows
print("\nFirst 5 rows (key columns):")
key_cols = ['I FGC(A)', 'B1 (T)', 'B2 (T)', 'B3 (T)', 'Angle (rad)']
key_cols = [c for c in key_cols if c in ref_df.columns]
display(ref_df[key_cols].head())

---
## 4. Load Kn Calibration Coefficients

The Kn coefficients encode the coil sensitivity for each harmonic order.
They depend on:
- Coil geometry (radius, turns, winding pattern)
- Compensation scheme (which coils are combined for ABS vs CMP channels)

**Important**: Use the Kn file that matches the compensation scheme in your measurement!

In [ ]:
# Load kn coefficients
kn = load_segment_kn_txt(kn_file)

print(f"Kn loaded from: {kn_file.name}")
print(f"  Harmonic orders: {list(kn.orders)}")
print(f"  Number of harmonics: {len(kn.orders)}")

# Display kn magnitudes
print("\nKn coefficient magnitudes:")
print(f"{'n':>3} {'|kn_abs|':>15} {'|kn_cmp|':>15} {'cmp/abs':>10}")
print("-" * 48)
for i, n in enumerate(kn.orders):
    abs_mag = np.abs(kn.kn_abs[i])
    cmp_mag = np.abs(kn.kn_cmp[i])
    ratio = cmp_mag / abs_mag if abs_mag > 1e-30 else np.nan
    print(f"{n:3d} {abs_mag:15.6e} {cmp_mag:15.6e} {ratio:10.4f}")

---
## 5. Parse Raw BTP8 Flux Data

### BTP8 File Format

The BTP8 flux files have 4 columns:
- Column 0: `df_abs` - Incremental flux from absolute channel (Wb)
- Column 1: `encoder` - Encoder position (counts)
- Column 2: `df_cmp` - Incremental flux from compensated channel (Wb)
- Column 3: `encoder` - Encoder position (duplicate)

Each row represents one sample (one angular position within a turn).

In [ ]:
def parse_btp8_flux_file(flux_path: Path) -> tuple:
    """Parse BTP8 flux file (4-column format).
    
    Returns:
        df_abs: Incremental flux, absolute channel (Wb)
        df_cmp: Incremental flux, compensated channel (Wb)
        encoder: Encoder counts (for timing)
    """
    data = np.loadtxt(flux_path)
    df_abs = data[:, 0]
    encoder = data[:, 1]
    df_cmp = data[:, 2]
    return df_abs, df_cmp, encoder


def parse_btp8_current_file(current_path: Path) -> np.ndarray:
    """Parse BTP8 current file (single column)."""
    return np.loadtxt(current_path)


def encoder_to_time(encoder: np.ndarray, shaft_rpm: float, 
                    encoder_res: int = 40000) -> np.ndarray:
    """Convert encoder counts to time in seconds.
    
    Args:
        encoder: Encoder counts array
        shaft_rpm: Rotation speed in RPM
        encoder_res: Encoder resolution (counts per revolution)
    """
    counts_per_second = shaft_rpm * encoder_res / 60.0
    return encoder / counts_per_second


print("Parser functions defined.")

In [ ]:
# Find all raw data files
flux_files = sorted(dataset_folder.glob("*_fluxes_Ascii.txt"))
current_files = sorted(dataset_folder.glob("*_current.txt"))

print(f"Found {len(flux_files)} flux files and {len(current_files)} current files")

if len(flux_files) == 0:
    raise FileNotFoundError("No flux files found in dataset folder")

# Show first few files
print("\nFirst 5 flux files:")
for f in flux_files[:5]:
    print(f"  {f.name}")

In [ ]:
# Inspect the first file to understand the data structure
test_flux_path = flux_files[0]
test_current_path = current_files[0]

df_abs, df_cmp, encoder = parse_btp8_flux_file(test_flux_path)
current = parse_btp8_current_file(test_current_path)
time = encoder_to_time(encoder, SHAFT_SPEED_RPM)

print(f"File: {test_flux_path.name}")
print(f"  Total samples: {len(df_abs)}")
print(f"  Samples per turn: {SAMPLES_PER_TURN}")
print(f"  Complete turns: {len(df_abs) // SAMPLES_PER_TURN}")
print(f"\nFlux ranges:")
print(f"  df_abs: [{df_abs.min():.6e}, {df_abs.max():.6e}] Wb")
print(f"  df_cmp: [{df_cmp.min():.6e}, {df_cmp.max():.6e}] Wb")

---
## 6. Process Data and Compute Harmonics

### Processing Pipeline Steps

1. **Parse** raw flux and current files
2. **Reshape** samples into turns (using SAMPLES_PER_TURN)
3. **Drift correction** (if enabled): Remove linear drift from integrated flux
4. **FFT**: Compute Fourier coefficients from flux
5. **Kn application**: Scale by coil sensitivity
6. **Rotation** (if enabled): Align phase to main field
7. **Merge**: Combine ABS and CMP channels appropriately

In [ ]:
def process_btp8_run(flux_path: Path, current_path: Path, kn,
                     samples_per_turn: int,
                     shaft_rpm: float,
                     magnet_order: int,
                     r_ref_m: float,
                     options: tuple):
    """Process a single BTP8 measurement run.
    
    This implements the same pipeline as the legacy C++ analyzer.
    
    Args:
        flux_path: Path to flux file
        current_path: Path to current file
        kn: SegmentKn calibration object
        samples_per_turn: Number of samples per revolution
        shaft_rpm: Rotation speed
        magnet_order: Main field order (m)
        r_ref_m: Reference radius for multipole normalization
        options: Tuple of enabled options ("dri", "rot", etc.)
    
    Returns:
        result: KnPerTurnResult with computed harmonics
        n_turns: Number of complete turns processed
    """
    # Parse files
    df_abs, df_cmp, encoder = parse_btp8_flux_file(flux_path)
    current = parse_btp8_current_file(current_path)
    time = encoder_to_time(encoder, shaft_rpm)
    
    # Align current to flux length (they may differ slightly)
    n_flux = len(df_abs)
    n_curr = len(current)
    if n_curr != n_flux:
        indices = np.linspace(0, n_curr - 1, n_flux).astype(int)
        current_aligned = current[indices]
    else:
        current_aligned = current
    
    # Truncate to complete turns only
    n_turns = n_flux // samples_per_turn
    n_samples = n_turns * samples_per_turn
    
    # Reshape into (n_turns, samples_per_turn)
    df_abs_turns = df_abs[:n_samples].reshape(n_turns, samples_per_turn)
    df_cmp_turns = df_cmp[:n_samples].reshape(n_turns, samples_per_turn)
    t_turns = time[:n_samples].reshape(n_turns, samples_per_turn)
    I_turns = current_aligned[:n_samples].reshape(n_turns, samples_per_turn)
    
    # Run the pipeline (this is the core computation)
    result = compute_legacy_kn_per_turn(
        df_abs_turns=df_abs_turns,
        df_cmp_turns=df_cmp_turns,
        t_turns=t_turns,
        I_turns=I_turns,
        kn=kn,
        Rref_m=r_ref_m,
        magnet_order=magnet_order,
        absCalib=1.0,
        options=options,
    )
    
    return result, n_turns


print("Processing function defined.")

In [ ]:
# Process all runs and collect results
print(f"Processing {len(flux_files)} measurement runs...")
print(f"Options: {OPTIONS}")
print(f"Samples per turn: {SAMPLES_PER_TURN}")
print()

all_results = []
total_turns = 0

for i, (flux_path, current_path) in enumerate(zip(flux_files, current_files)):
    result, n_turns = process_btp8_run(
        flux_path=flux_path,
        current_path=current_path,
        kn=kn,
        samples_per_turn=SAMPLES_PER_TURN,
        shaft_rpm=SHAFT_SPEED_RPM,
        magnet_order=MAGNET_ORDER,
        r_ref_m=R_REF_M,
        options=OPTIONS,
    )
    all_results.append(result)
    total_turns += n_turns
    
    if (i + 1) % 10 == 0:
        print(f"  Processed {i + 1}/{len(flux_files)} runs...")

print(f"\nDone! Processed {total_turns} total turns from {len(flux_files)} runs.")
print(f"Reference has {len(ref_df)} turns.")

---
## 7. Compare with Reference

Now we compare our computed harmonics with the golden reference values.

In [ ]:
# Build a DataFrame with all computed harmonics
rows = []
turn_idx = 0

for result in all_results:
    # Merge coefficients: ABS for n<=m, CMP for n>m
    C_merged, _ = merge_coefficients(
        C_abs=result.C_abs,
        C_cmp=result.C_cmp,
        magnet_order=MAGNET_ORDER,
        mode="abs_upto_m_cmp_above",
    )
    
    for t in range(result.C_abs.shape[0]):
        row = {"turn_idx": turn_idx}
        for i, n in enumerate(result.orders):
            row[f"B{n}"] = np.real(C_merged[t, i])
            row[f"A{n}"] = np.imag(C_merged[t, i])
        rows.append(row)
        turn_idx += 1

computed_df = pd.DataFrame(rows)
print(f"Computed results: {len(computed_df)} turns")
print(f"Reference results: {len(ref_df)} turns")

In [ ]:
# Compare harmonics turn by turn
n_compare = min(len(computed_df), len(ref_df))
print(f"Comparing first {n_compare} turns...\n")

print("=" * 80)
print("HARMONIC COMPARISON SUMMARY")
print("=" * 80)
print(f"{'n':>3} {'Max |Diff|':>15} {'Max |Rel Diff|':>15} {'Status':>12}")
print("-" * 50)

comparison_results = []

for n in range(1, 16):
    comp_col = f"B{n}"
    ref_col = f"B{n} (T)"
    
    if comp_col not in computed_df.columns or ref_col not in ref_df.columns:
        continue
    
    comp_vals = computed_df[comp_col].values[:n_compare]
    ref_vals = ref_df[ref_col].values[:n_compare]
    
    # Absolute difference
    abs_diff = np.abs(comp_vals - ref_vals)
    max_abs_diff = np.max(abs_diff)
    
    # Relative difference (avoid division by zero)
    with np.errstate(divide='ignore', invalid='ignore'):
        rel_diff = np.abs((comp_vals - ref_vals) / ref_vals)
        rel_diff = np.where(np.isfinite(rel_diff), rel_diff, 0)
    max_rel_diff = np.max(rel_diff)
    
    # Determine status
    if max_rel_diff < 1e-6:
        status = "EXCELLENT"
    elif max_rel_diff < 1e-3:
        status = "GOOD"
    elif max_rel_diff < 0.1:
        status = "CLOSE"
    else:
        status = "MISMATCH"
    
    print(f"{n:3d} {max_abs_diff:15.6e} {max_rel_diff:15.6e} {status:>12}")
    
    comparison_results.append({
        "n": n,
        "max_abs_diff": max_abs_diff,
        "max_rel_diff": max_rel_diff,
        "status": status,
    })

In [ ]:
# Detailed comparison for main field (B2 for quadrupole)
main_n = MAGNET_ORDER
comp_col = f"B{main_n}"
ref_col = f"B{main_n} (T)"

print(f"\nDetailed comparison for main field (n={main_n}):")
print(f"{'Turn':>6} {'Computed':>16} {'Reference':>16} {'Diff':>16} {'Rel Diff':>12}")
print("-" * 70)

for i in range(min(10, n_compare)):
    comp_val = computed_df[comp_col].iloc[i]
    ref_val = ref_df[ref_col].iloc[i]
    diff = comp_val - ref_val
    rel_diff = diff / ref_val if abs(ref_val) > 1e-20 else 0
    print(f"{i:6d} {comp_val:16.9e} {ref_val:16.9e} {diff:16.9e} {rel_diff:12.6e}")

---
## 8. Visualization

In [ ]:
# Plot computed vs reference for main field
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# B2 time series
ax = axes[0, 0]
ax.plot(ref_df[f"B{MAGNET_ORDER} (T)"].values[:n_compare], 'b-', label='Reference', alpha=0.7)
ax.plot(computed_df[f"B{MAGNET_ORDER}"].values[:n_compare], 'r--', label='Computed', alpha=0.7)
ax.set_xlabel('Turn')
ax.set_ylabel(f'B{MAGNET_ORDER} (T)')
ax.set_title(f'Main Field (n={MAGNET_ORDER}): Time Series')
ax.legend()
ax.grid(True, alpha=0.3)

# B2 scatter (computed vs reference)
ax = axes[0, 1]
ref_vals = ref_df[f"B{MAGNET_ORDER} (T)"].values[:n_compare]
comp_vals = computed_df[f"B{MAGNET_ORDER}"].values[:n_compare]
ax.scatter(ref_vals, comp_vals, alpha=0.5, s=10)
ax.plot([ref_vals.min(), ref_vals.max()], [ref_vals.min(), ref_vals.max()], 'k--', label='Perfect match')
ax.set_xlabel('Reference (T)')
ax.set_ylabel('Computed (T)')
ax.set_title(f'B{MAGNET_ORDER}: Computed vs Reference')
ax.legend()
ax.grid(True, alpha=0.3)

# Difference histogram
ax = axes[1, 0]
diff = comp_vals - ref_vals
ax.hist(diff, bins=50, edgecolor='black', alpha=0.7)
ax.axvline(0, color='r', linestyle='--', label='Zero')
ax.set_xlabel('Difference (T)')
ax.set_ylabel('Count')
ax.set_title(f'B{MAGNET_ORDER}: Difference Distribution')
ax.legend()
ax.grid(True, alpha=0.3)

# Relative difference by harmonic
ax = axes[1, 1]
harmonics = [r['n'] for r in comparison_results]
rel_diffs = [r['max_rel_diff'] for r in comparison_results]
colors = ['green' if d < 1e-3 else 'orange' if d < 0.1 else 'red' for d in rel_diffs]
ax.bar(harmonics, rel_diffs, color=colors, edgecolor='black')
ax.axhline(1e-3, color='g', linestyle='--', label='Good threshold')
ax.axhline(0.1, color='orange', linestyle='--', label='Close threshold')
ax.set_xlabel('Harmonic Order n')
ax.set_ylabel('Max Relative Difference')
ax.set_title('Parity Check by Harmonic')
ax.set_yscale('log')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'parity_comparison.png', dpi=150)
plt.show()

print(f"\nPlot saved to: {output_dir / 'parity_comparison.png'}")

---
## 9. How to Replicate Using the GUI

This section explains step-by-step how to reproduce these results using the graphical user interface.

### Step 1: Launch the GUI

```bash
cd rotating_coil_analyzer
python -m rotating_coil_analyzer.gui.app
```

### Step 2: File Browser Tab

1. Navigate to your dataset folder (e.g., `golden_standards/golden_standard_01_LIU_BTP8/Central/20190718_141359_LIU`)
2. The file browser will show all flux files, current files, and results files
3. Select the files you want to analyze

### Step 3: Run Preview Tab

1. Load a flux file to preview the raw data
2. Verify the data looks correct (no obvious artifacts)
3. Check that the number of samples matches expectations

### Step 4: Coil Calibration Tab

1. **Load Kn File**: Click "Load Kn TXT" and select the appropriate Kn file:
   - For BD_AE compensation: `Kn_DQ_5_18_7_250_47x50_0001_BD_AE.txt`
   - For A_ABCD compensation: `Kn_DQ_5_18_7_250_47x50_0001_A_ABCD.txt`

2. **Alternative - Compute from Head CSV**:
   - Load the measurement head geometry CSV
   - Select the absolute and compensated channel connections
   - Click "Compute Kn" to generate calibration coefficients

3. Verify the Kn values are loaded correctly (check the table display)

### Step 5: Harmonic Merge Tab

1. **Set Magnet Order**: Enter `2` for quadrupole
2. **Set Reference Radius**: Enter `1.0` m (or as specified in Parameters.txt)
3. **Select Merge Mode**: Choose "ABS up to m, CMP above"
   - This uses ABS channel for n ≤ 2 (main field)
   - Uses CMP channel for n > 2 (error harmonics)
4. **Set Compensation Scheme**: Enter "BD_AE" (for documentation)

### Step 6: Plots Tab

1. **Set Analysis Options**:
   - Enable "Drift Correction" (dri)
   - Enable "Rotation" (rot)
   - Disable other options unless needed

2. **Set Samples Per Turn**: Enter `512` (critical for BTP8 format!)

3. **Run Analysis**: Click "Compute" to process all selected runs

4. **View Results**: The plots will show the computed harmonics

5. **Export**: Click "Export CSV" to save results in the standard format

### Verification

Compare your exported CSV with the reference results file:
- B2 values should match to within 0.1%
- Higher harmonics (B3-B15) should match to within a few percent
- Any large discrepancies indicate a configuration mismatch

---
## 10. Export Results

In [ ]:
# Export computed results
computed_path = output_dir / "computed_results.csv"
computed_df.to_csv(computed_path, index=False)
print(f"Computed results saved to: {computed_path}")

# Export comparison summary
comparison_df = pd.DataFrame(comparison_results)
comparison_path = output_dir / "comparison_summary.csv"
comparison_df.to_csv(comparison_path, index=False)
print(f"Comparison summary saved to: {comparison_path}")

# Export provenance metadata
metadata = {
    "timestamp": datetime.now(timezone.utc).isoformat(),
    "dataset_folder": str(dataset_folder),
    "kn_file": str(kn_file),
    "magnet_order": MAGNET_ORDER,
    "r_ref_m": R_REF_M,
    "samples_per_turn": SAMPLES_PER_TURN,
    "options": OPTIONS,
    "n_computed_turns": len(computed_df),
    "n_reference_turns": len(ref_df),
}

metadata_path = output_dir / "provenance.json"
with open(metadata_path, "w") as f:
    json.dump(metadata, f, indent=2)
print(f"Provenance metadata saved to: {metadata_path}")

---
## 11. Final Summary

In [ ]:
# Print final summary
print("=" * 80)
print("GOLDEN STANDARD PARITY VALIDATION - FINAL REPORT")
print("=" * 80)
print(f"\nGenerated: {datetime.now().isoformat()}")
print(f"\nDataset: {dataset_folder.name}")
print(f"Kn file: {kn_file.name}")
print(f"\nConfiguration:")
print(f"  Magnet order: {MAGNET_ORDER}")
print(f"  Reference radius: {R_REF_M} m")
print(f"  Samples per turn: {SAMPLES_PER_TURN}")
print(f"  Options: {OPTIONS}")

print(f"\nData processed:")
print(f"  Computed turns: {len(computed_df)}")
print(f"  Reference turns: {len(ref_df)}")
print(f"  Compared turns: {n_compare}")

print(f"\nParity Results:")
excellent = sum(1 for r in comparison_results if r['status'] == 'EXCELLENT')
good = sum(1 for r in comparison_results if r['status'] == 'GOOD')
close = sum(1 for r in comparison_results if r['status'] == 'CLOSE')
mismatch = sum(1 for r in comparison_results if r['status'] == 'MISMATCH')

print(f"  EXCELLENT (< 1e-6 rel): {excellent} harmonics")
print(f"  GOOD (< 1e-3 rel):      {good} harmonics")
print(f"  CLOSE (< 0.1 rel):      {close} harmonics")
print(f"  MISMATCH (>= 0.1 rel):  {mismatch} harmonics")

if mismatch == 0:
    print("\n" + "*" * 80)
    print("VALIDATION PASSED: All harmonics match within acceptable tolerance!")
    print("*" * 80)
else:
    print("\n" + "!" * 80)
    print(f"VALIDATION WARNING: {mismatch} harmonics have significant mismatch.")
    print("Review the comparison details above for debugging.")
    print("!" * 80)

print(f"\nOutput files:")
print(f"  {computed_path}")
print(f"  {comparison_path}")
print(f"  {metadata_path}")
print(f"  {output_dir / 'parity_comparison.png'}")